## Vector Search

In [7]:
import json
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer

In [4]:
documents_json_path = "/workspaces/llmzoomcamp/01-intro/documents.json"

with open(documents_json_path, 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Step 2: Create Embeddings using Pretrained Models

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [6]:
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/python/3.12.1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
len(model.encode("This is a simple sentence"))

768

In [9]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

### Step 3: Setup ElasticSearch connection 

In [15]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '529efc25fa99', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'bOjaIQjLTXidrqTiPS3mOw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Step 4: Create Mappings and Index
* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored 

In [17]:
# Define the settings and mappings for the Elasticsearch index
index_settings = {
    "settings": {
        # Number of primary shards for the index
        "number_of_shards": 1,
        # Number of replica shards for each primary shard
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            # Field for storing text data
            "text": {
                "type": "text"  # Analyzed text field for full-text search
            },
            # Field for storing section information
            "section": {
                "type": "text"  # Analyzed text field for full-text search
            },
            # Field for storing question text
            "question": {
                "type": "text"  # Analyzed text field for full-text search
            },
            # Field for storing course information, treated as a keyword (not analyzed)
            "course": {
                "type": "keyword"  # Exact value field, suitable for filtering and aggregations
            },
            # Field for storing dense vector representations of text, useful for similarity search
            "text_vector": {
                "type": "dense_vector",  # Field type for dense vectors
                "dims": 768,  # Number of dimensions in the vector
                "index": True,  # Enable indexing for similarity search
                "similarity": "cosine"  # Similarity metric used for vector comparisons
            },
        }
    }
}


In [18]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Step 5: Add documents into index

In [19]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

### Step 6: Create end user query

In [20]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [21]:
# Define the vector search query
query = {
    # The field containing the dense vector representation
    "field": "text_vector",
    # The vector to search for similarity against the vectors in the index
    "query_vector": vector_search_term,
    # The number of top matching results to return (k nearest neighbors)
    "k": 5,
    # The number of candidate vectors to consider for the search
    "num_candidates": 10000,  # Search over a large number of candidates to ensure accurate results
}


In [29]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'WKp_CpIBr-oIE_1B5OMt',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'a6qACpIBr-oIE_1BFubB',
  '_score': 0.61347336,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

### Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [23]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [30]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    source=["text", "section", "question", "course"],
    size=5
)

In [ ]:
response["hits"]["hits"]

In [32]:
print("End")

End
